<a href="https://colab.research.google.com/github/rishik-00/Natural-Language-Processing/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some basic code to get used to it

In [0]:
import tensorflow as tf
import json

In [3]:
from tensorflow.keras.preprocessing.text import  Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentences = [
             'i love my dogs',
             'i love my cats',
             'Do you love my dogs?',
             'I know my dog is amazing'
]
tokenizer = Tokenizer(num_words = 100, oov_token = 'OOV')
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding = 'post')
print(word_index)
print(sequences)

print('padded :', padded)

{'OOV': 1, 'my': 2, 'i': 3, 'love': 4, 'dogs': 5, 'cats': 6, 'do': 7, 'you': 8, 'know': 9, 'dog': 10, 'is': 11, 'amazing': 12}
[[3, 4, 2, 5], [3, 4, 2, 6], [7, 8, 4, 2, 5], [3, 9, 2, 10, 11, 12]]
padded : [[ 3  4  2  5  0  0]
 [ 3  4  2  6  0  0]
 [ 7  8  4  2  5  0]
 [ 3  9  2 10 11 12]]


In [12]:
test = ['you know what i love my dog',
        'I am kidding my cat is amazing']

sequences = tokenizer.texts_to_sequences(test)
padded = pad_sequences(sequences, maxlen = 6)
print(sequences)
print('padded : ', padded)

[[8, 9, 1, 3, 4, 2, 10], [3, 1, 1, 2, 1, 11, 12]]
padded :  [[ 9  1  3  4  2 10]
 [ 1  1  2  1 11 12]]


# Using IMDB dataset

In [4]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVT37BJ/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVT37BJ/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVT37BJ/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

? ? ? ? ? ? ? b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4955 - accuracy: 0.7443 - val_loss: 0.3481 - val_accuracy: 0.8467
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2353 - accuracy: 0.9090 - val_loss: 0.3784 - val_accuracy: 0.8367
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0860 - accuracy: 0.9790 - val_loss: 0.4661 - val_accuracy: 0.8240
Epoch 4/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0214 - accuracy: 0.9970 - val_loss: 0.5281 - val_accuracy: 0.8235
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0052 - accuracy: 0.9997 - val_loss: 0.5893 - val_accuracy: 0.8238
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6371 - val_accuracy: 0.8264
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 8.8871e-04 - accuracy: 1.0000 - val_loss: 0.6818 - val_accuracy: 0.8260
Ep

In [14]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [17]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]


# Importing the subset imdb data

In [19]:
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete15VG1Y/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete15VG1Y/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incomplete15VG1Y/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_data, test_data = imdb['train'], imdb['test']
train_data = train_data.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))
test_data = test_data.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

In [0]:
tokenizer = info.features['text'].encoder

In [34]:
print(tokenizer.subwords)

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_', 't_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', ' (', 'are_', 'his_', 'have_', 'film_', 'not_', 'ing_', 'be_', 'ed_', 'you_', ' "', 'it', 'd_', 'an_', 'he_', 'by_', 'at_', 'one_', 'who_', 'y_', 'from_', 'e_', 'or_', 'all_', 'like_', 'they_', '" ', 'so_', 'just_', 'has_', ') ', 'her_', 'about_', 'out_', 'This_', 'some_', 'ly_', 'movie', 'film', 'very_', 'more_', 'It_', 'would_', 'what_', 'when_', 'which_', 'good_', 'if_', 'up_', 'only_', 'even_', 'their_', 'had_', 'really_', 'my_', 'can_', 'no_', 'were_', 'see_', 'she_', '? ', 'than_', '! ', 'there_', 'get_', 'been_', 'into_', ' - ', 'will_', 'much_', 'story_', 'because_', 'ing', 'time_', 'n_', 'we_', 'ed', 'me_', ': ', 'most_', 'other_', 'don', 'do_', 'm_', 'es_', 'how_', 'also_', 'make_', 'its_', 'could_', 'first_', 'any_', "' ", 'people_', 'great_', 've_', 'ly', 'er_', 'made_', 'r_', 'B

In [35]:
sample = 'Taarak mehta ka olta chashmah'
tokenized_string = tokenizer.encode(sample)
print('tokenizer string : ', tokenized_string)

original_string = tokenizer.decode(tokenized_string)
print(' original string : ', original_string)

tokenizer string :  [817, 2439, 231, 220, 2331, 4, 5333, 773, 1848, 7850, 3029, 2046]
 original string :  Taarak mehta ka olta chashmah


In [46]:
embedding_dim = 64
model = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
                                    tf.keras.layers.GlobalAveragePooling1D(),
                                    tf.keras.layers.Dense(6, activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation = 'sigmoid'),
                                    
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
global_average_pooling1d_5 ( (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 390       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 7         
Total params: 524,237
Trainable params: 524,237
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [48]:
history = model.fit(train_data, epochs = num_epochs, validation_data= test_data)

Epoch 1/10
25000/25000 [==============================] - 276s 11ms/step - loss: 0.6862 - accuracy: 0.5246 - val_loss: 0.6864 - val_accuracy: 0.5321
Epoch 2/10
25000/25000 [==============================] - 285s 11ms/step - loss: 0.6837 - accuracy: 0.5347 - val_loss: 0.6857 - val_accuracy: 0.5348
Epoch 3/10
25000/25000 [==============================] - 283s 11ms/step - loss: 0.6828 - accuracy: 0.5375 - val_loss: 0.6857 - val_accuracy: 0.5371
Epoch 4/10
25000/25000 [==============================] - 287s 11ms/step - loss: 0.6820 - accuracy: 0.5392 - val_loss: 0.6861 - val_accuracy: 0.5367
Epoch 5/10
25000/25000 [==============================] - 282s 11ms/step - loss: 0.6815 - accuracy: 0.5405 - val_loss: 0.6865 - val_accuracy: 0.5355
Epoch 6/10
25000/25000 [==============================] - 283s 11ms/step - loss: 0.6812 - accuracy: 0.5412 - val_loss: 0.6869 - val_accuracy: 0.5343
Epoch 7/10
25000/25000 [==============================] - 292s 12ms/step - loss: 0.6810 - accuracy: 0.5411

In [39]:
train_data

<DatasetV1Adapter shapes: ((None,), ()), types: (tf.int64, tf.int64)>